In [41]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore, db
import json

In [42]:
#cred = credentials.Certificate('/Users/evanrussek/forage_jsp/analysis/foragetask-firebase-private-key.json')
cred = credentials.Certificate('C:\\Users\\erussek\\forage_jsp\\analysis\\foragetask-firebase-private-key.json')
default_app = firebase_admin.initialize_app(cred)
client = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [59]:
dfs = []
bonus_df = pd.DataFrame()
for subj in client.collection('foragetask').document('run5').collection('subjects').stream():
    Qdfs = []
    subj_dfs = []
    taskdata_collection = client.collection('foragetask/run5/subjects/{0}/taskdata'.format(subj.id)).stream()
    has_end = False # did this subject finish the task?\n",
    has_start = False
    subjectID = subj.id
    for tc in taskdata_collection:
        if tc.id == "start":
            has_start = True
            start_dict = tc.to_dict()
            subjectID = start_dict['subjectID']
        elif tc.id == "end":
            print(subj.id) # ok. it's loopint through this too many times...............
            has_end = True
            end_dict = tc.to_dict()
            end_time = end_dict['end_time']
            bonus_points = end_dict['bonus_points'][0]
        elif tc.id == "Q_data":
            has_start = True
            Q_dict = tc.to_dict()
            Q_data_json = json.loads(Q_dict['trial_data'])
            Q_df = pd.DataFrame(Q_data_json)
            Q_df['subjectID'] = subj.id
            Qdfs.append(Q_df)
        else:
            trial_dict = tc.to_dict()
            trial_name = tc.id
            trial_data_json = json.loads(trial_dict['trial_data'])
            trial_df = pd.DataFrame(trial_data_json)
            trial_df['subjectID'] = subjectID
            if has_start:
                trial_df['subjectID'] = subjectID
                #trial_df['date'] = date
            else:
                trial_df['subjectID'] = None
                #trial_df['date'] = None
            if has_end:
                trial_df['bonus_points'] = bonus_points
            else:
                trial_df['bonus_points'] = None
            subj_dfs.append(trial_df)
    if (has_end & (len(str(subjectID)) > 20)):
        bonus_dict = {'subjectID': subjectID, 'bonus_points': bonus_points}
        bonus_df = bonus_df.append(bonus_dict, ignore_index = True)
        dfs = dfs + subj_dfs
combined_df = pd.concat(dfs, sort = True)
combined_df.to_csv('data/run5_data.csv')

0tz8Cw7eVkTKuO2UHapNknfteQJ2
GSZ78YOlHoNsKOhOebWzTlFn1qt1
HONcxHvqn2hQGdSlCE8rccN5vW73
IqlID2KvgGR4bB4BU2TdYby5PC23
LXSGIiGuJtNx686E7Tbo3itaOoE3
PjboZsy6D1dVFR4AgyTSCBkFjQz2
QUgzjuKjOXXI5Oo3fn9gvm1odW53
SSqxANlQQHe4i1b8f6obWRiyBcz1
Vqgfw0lFe5PTaYVKnZJAn8rawdI2
WEiZLxNmo8WhAnvO0IlgBwtC1Nv2
Xmf0up8ofRUbwnSLToRvsNBV21E2
ZPps9gBHm3OklDx3X9O4sPnPkUz1
eT4Ilf2EBcYJtedOlfpnYzWbApp1
rRJWJvy7FfXbeci2KSNhvKjUtPv1
tExFRQwOYPWyadFLvlsdsfjk3I12
teFB7720FGaPsRVqky7AijJldoO2
uV8cFS1RBXPE2KrDw2M31RkQeTz2
wfLELWIiOFbi49GWsZquujSrppm1
yasOSuObuxZ7i9is7TdaJrqngSh2
z6kIoIOWfld8M7bfNRqBcRK4RSX2


In [58]:
len(dfs)

120

In [49]:
def compute_bonus(row):
    pct = np.min([np.max([row['bonus_points'] / (30000 - 0),0]),1])
    return np.around(2.00*pct, decimals = 2)
bonus_df['bonus'] = bonus_df.apply(compute_bonus, axis = 1)

In [50]:
bonus_df
bonus_df[['subjectID', 'bonus']].to_csv('bonus_payment_files\\run5_bonus_payments.csv', sep = ',', header = False, index = False)

In [33]:
bonus_dict

{'subjectID': '5c38a39645529a0001226c91', 'bonus_points': 13810}

In [51]:
bonus_df

,bonus_points,subjectID,bonus
0,20879.0,5accded9a3ba7a0001b504e5,1.39
1,20925.0,5af33167dd84460001c1dfcf,1.40
2,11100.0,5d7fdc371c09e60001dd48f5,0.74
3,32125.0,581ccd016c73180001fa5b2a,2.00
4,20622.0,5cc0c8b64ded64000152d66e,1.37
5,12039.0,5c116d14ce422b0001d67607,0.80
6,14320.0,5dcdc93b4f433409300aa8c8,0.95
7,15094.0,59dcba5fac0af70001b36245,1.01
8,14614.0,5755c957eb80c4000741a9ce,0.97
9,13060.0,58343680ff446c00014fb3e6,0.87
